In [4]:
import numpy as np
import pandas as pd

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
data.head()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age,diabetes
0,316,2,112,68,22,94,34.1,0.315,26,0
1,25,11,143,94,33,146,36.6,0.254,51,1
2,710,2,93,64,32,160,38.0,0.674,23,1
3,658,1,120,80,48,200,38.9,1.162,41,0
4,542,3,128,72,25,190,32.4,0.549,27,1


In [5]:
data.drop('p_id',axis=1,inplace = True)
test_ids = test["p_id"]

x = data.drop('diabetes',axis=1)
y = data.diabetes

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=30)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

In [12]:
model = LogisticRegression()
model = tree.DecisionTreeClassifier()
model = SVC()
model = RandomForestRegressor(n_estimators = 100, random_state = 0)

model.fit(x_train, y_train)
model.score(x_test, y_test)

test.head()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age
0,437,12,140,85,33,0,37.4,0.244,41
1,411,6,102,90,39,0,35.7,0.674,28
2,639,7,97,76,32,91,40.9,0.871,32
3,213,7,179,95,31,0,34.2,0.164,60
4,181,6,87,80,0,0,23.2,0.084,32


In [13]:
p=test.drop(['p_id'],axis=1)
rec = model.predict(p)
df = pd.DataFrame({"p_id": test_ids.values,
                   "diabetes":rec,
                  })
df.to_csv("submission.csv", index=False)